In [1]:
import pandas as pd

df = pd.read_csv(r'C:\Users\Hp\Documents\Music Recommender\Data\spotify_millsongdata.csv')
df.head()

artist                   song                                        link  \
0   ABBA  Ahe's My Kind Of Girl  /a/abba/ahes+my+kind+of+girl_20598417.html   
1   ABBA       Andante, Andante       /a/abba/andante+andante_20002708.html   
2   ABBA         As Good As New        /a/abba/as+good+as+new_20003033.html   
3   ABBA                   Bang                  /a/abba/bang_20598415.html   
4   ABBA       Bang-A-Boomerang      /a/abba/bang+a+boomerang_20002668.html   

                                                text  
0  Look at her face, it's a wonderful face  \r\nA...  
1  Take it easy with me, please  \r\nTouch me gen...  
2  I'll never know why I had to go  \r\nWhy I had...  
3  Making somebody happy is a question of give an...  
4  Making somebody happy is a question of give an...

In [2]:
df.shape

(57650, 4)

In [3]:
df = df.sample(7650).drop('link', axis=1).reset_index(drop=True)
df.head()

artist                   song  \
0            HIM          Fade Into You   
1        Extreme      When Will It Rain   
2     Carpenters     Sweet, Sweet Smile   
3    Moody Blues   Watching And Waiting   
4  Grateful Dead  Satisfaction (encore)   

                                                text  
0  I want to hold the hand inside you  \r\nI want...  
1  No one ever sees you cry  \r\nYou just keep it...  
2  You're always in my heart  \r\nFrom early in t...  
3  Watching and waiting  \r\nFor a friend to play...  
4  I can't get no, satisfaction. I can't get no s...

In [4]:
df.shape

(7650, 3)

TEXT CLEANING

In [5]:
df['text'] = df['text'].str.lower().replace(r'\w\s', '').replace(r'\n', '', regex = True).replace(r'\r', '', regex=True)
df['text']

0       i want to hold the hand inside you  i want to ...
1       no one ever sees you cry  you just keep it all...
2       you're always in my heart  from early in the m...
3       watching and waiting  for a friend to play wit...
4       i can't get no, satisfaction. i can't get no s...
                              ...                        
7645    i have never known a love like this,  before i...
7646    deck the halls with boughs of holly  fa-la-la-...
7647    i was angry with my friend  i told him so and ...
7648    the way that sunlight flits across your skirt,...
7649    down along the cove,  i spied my true love com...
Name: text, Length: 7650, dtype: object

In [6]:
df.head()

artist                   song  \
0            HIM          Fade Into You   
1        Extreme      When Will It Rain   
2     Carpenters     Sweet, Sweet Smile   
3    Moody Blues   Watching And Waiting   
4  Grateful Dead  Satisfaction (encore)   

                                                text  
0  i want to hold the hand inside you  i want to ...  
1  no one ever sees you cry  you just keep it all...  
2  you're always in my heart  from early in the m...  
3  watching and waiting  for a friend to play wit...  
4  i can't get no, satisfaction. i can't get no s...

In [7]:
import nltk
from nltk.stem.porter import PorterStemmer

In [8]:
stemming = PorterStemmer()

In [9]:
def word_token(txt):
    words = nltk.word_tokenize(txt)
    stem_words = [stemming.stem(w) for w in words]
    return " ".join(stem_words)

In [10]:
word_token("the sun is rising in the east")

'the sun is rise in the east'

In [11]:
df['text'].apply(lambda x:word_token(x))

0       i want to hold the hand insid you i want to ta...
1       no one ever see you cri you just keep it all i...
2       you 're alway in my heart from earli in the mo...
3       watch and wait for a friend to play with whi h...
4       i ca n't get no , satisfact . i ca n't get no ...
                              ...                        
7645    i have never known a love like thi , befor in ...
7646    deck the hall with bough of holli fa-la-la-la-...
7647    i wa angri with my friend i told him so and my...
7648    the way that sunlight flit across your skirt ,...
7649    down along the cove , i spi my true love comin...
Name: text, Length: 7650, dtype: object

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [14]:
matrix = tfidf.fit_transform(df['text'])

In [15]:
similarity = cosine_similarity(matrix)

In [16]:
len(similarity[1])

7650

In [17]:
list(enumerate(similarity[1]))

[(0, 0.013413028871413067),
 (1, 1.0000000000000002),
 (2, 0.0014397460573600386),
 (3, 0.015061985976898496),
 (4, 0.0),
 (5, 0.008454279812063332),
 (6, 0.027050323020449147),
 (7, 0.01425129463128674),
 (8, 0.027938033585985126),
 (9, 0.0),
 (10, 0.01323622341679505),
 (11, 0.01809590065180466),
 (12, 0.009942276933529897),
 (13, 0.08460977866910627),
 (14, 0.01692430525551266),
 (15, 0.013808950288123083),
 (16, 0.037072804507269984),
 (17, 0.01989188356597657),
 (18, 0.017828502572136255),
 (19, 0.07258941061020714),
 (20, 0.0013850545322026152),
 (21, 0.0023046318109214412),
 (22, 0.0005025766681335724),
 (23, 0.0),
 (24, 0.003110483740403062),
 (25, 0.0),
 (26, 0.0035457351475316244),
 (27, 0.07378523329424216),
 (28, 0.001226596888908442),
 (29, 0.011965945303893187),
 (30, 0.003830027598928447),
 (31, 0.004741646468781172),
 (32, 0.03903266394902734),
 (33, 0.02100512296474949),
 (34, 0.011216798249748885),
 (35, 0.020516289905474987),
 (36, 0.006006010970235738),
 (37, 0.0076

RECOMMENDER SYSTEM

In [18]:
def recommender(song):
    idx = df[df['song'] == song].index[0]
    distance = sorted(list(enumerate(similarity[idx])), reverse=True, key = lambda x:x[1])
    similar_songs = []
    for s_id in distance[1:6]:
        similar_songs.append(df.iloc[s_id[0]].song)
    return similar_songs

In [19]:
recommender("So This Is Love?")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
df.tail()

artist                song  \
7645            ZZ Top          Prettyhead   
7646   Robbie Williams    Baby Girl Window   
7647         Van Halen    So This Is Love?   
7648          Maroon 5  Sleepy Windbreaker   
7649  Gordon Lightfoot        Endless Wire   

                                                   text  
7645  don't send me no bag, don't send me no skag  a...  
7646  i've got you rainbows seen your sunbeams  i kn...  
7647  oh, yeah  check me out on the good side here  ...  
7648  we filled the day with careless displays  goin...  
7649  i ain't got nothin' to hide  my mind is an end...

In [20]:
import pickle

In [26]:
pickle.dump(similarity, open(r'C:\Users\Hp\Documents\Music Recommender\similarity.pkl', "wb"))

In [27]:
pickle.dump(df, open(r'C:\Users\Hp\Documents\Music Recommender\data.pkl', "wb"))